# **Medical Q&A with RAG**

### Dr. Santosh Chapaneri
### Lead AI Product Engineer, Wolters Kluwer


In [1]:
from http.client import responses
!pip install \
langchain==0.3.2 \
langchain-community==0.3.1 \
langchain-core==0.3.63 \
langchain-text-splitters==0.3.8 \
langchain_huggingface==0.2.0 \
langchain_mistralai==0.2.10 \
langsmith==0.1.147 \
chromadb==0.5.5 \
pypdf \
faiss-cpu

  Using cached langchain-0.3.2-py3-none-any.whl (1.0 MB)
  Using cached langchain_community-0.3.1-py3-none-any.whl (2.4 MB)
  Using cached langchain_core-0.3.63-py3-none-any.whl (438 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl (32 kB)
  Using cached langchain_huggingface-0.2.0-py3-none-any.whl (27 kB)
  Using cached langchain_mistralai-0.2.10-py3-none-any.whl (16 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Using cached chromadb-0.5.5-py3-none-any.whl (584 kB)
                                              0.0/329.1 kB ? eta -:--:--
     ---                                   30.7/329.1 kB 660.6 kB/s eta 0:00:01
     -----                                 51.2/329.1 kB 871.5 kB/s eta 0:00:01
     -----                                 51.2/329.1 kB 871.5 kB/s eta 0:00:01
     -----                                 51.2/329.1 kB 871.5 kB/s eta 0:00:01
     -----                                 51.2/329.1 kB 871.5 kB/s eta 0:00:01
     -----         

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-prebuilt 1.0.7 requires langchain-core>=1.0.0, but you have langchain-core 0.3.63 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
cd ../../data/Medical_data/

E:\AIML Workshop\data\Medical_data


# **Login**

In [8]:
import os

os.environ["MISTRAL_API_KEY"] = "UV2oAjPsJrHBmTbVZJe5Id50iZEsNyMd"

# **Load documents**

In [9]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
def load_docs(directory):
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    return documents

In [13]:
directory = "../../data/Medical_data/"

documents = load_docs(directory)

len(documents)

36

In [15]:
for doc in documents[:3]:
    print("Document text:")
    print(doc.page_content)
    print("Document metadata:")
    print(doc.metadata)
    print("--"*100)

Document text:
 
 
 
 
Introduction to Asthma 
Childhood asthma can be controlled through education and treatment. Working as a team, school 
personnel, healthcare providers and parents can help children with asthma participate fully in school, 
sports and home activities.  
What is Asthma?  
Asthma is one of the most common chronic diseases o f 
childhood, affecting more than 6 million children. 
Asthma is a chronic inflammatory lung disease that can 
cause repeated episodes of cough, wheezing and brea thing 
difficulty. 
During an acute asthma episode, the airway lining in the 
lungs becomes inflamed and swollen. In addition, mu cus 
production occurs in the airway and muscles surroun ding 
the airway spasm. Combined, these cause a reduction  in air 
flow. 
Asthma is characterized by: 
• Airway inflammation: The airway lining becomes 
red, swollen, and narrow. 
• Airway obstruction: The muscles encircling the 
airway tighten causing the airway to narrow making 
it difficult to get ai

## **Split documents**

In [16]:
# split the documents into chunks
def split_docs(documents, chunk_size=512, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap,
      separators=["\n\n", "\n", " ", ""]
      )
    docs = text_splitter.split_documents(documents)
    return docs

In [17]:
docs = split_docs(documents)

print(len(docs))

331


In [18]:
for doc in docs[:5]:
    print("Document text:")
    print(doc.page_content)
    print("Document metadata:")
    print(doc.metadata)
    print("--"*100)

Document text:
Introduction to Asthma 
Childhood asthma can be controlled through education and treatment. Working as a team, school 
personnel, healthcare providers and parents can help children with asthma participate fully in school, 
sports and home activities.  
What is Asthma?  
Asthma is one of the most common chronic diseases o f 
childhood, affecting more than 6 million children. 
Asthma is a chronic inflammatory lung disease that can 
cause repeated episodes of cough, wheezing and brea thing
Document metadata:
{'source': '..\\..\\data\\Medical_data\\asthma.pdf', 'page': 0}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Document text:
difficulty. 
During an acute asthma episode, the airway lining in the 
lungs becomes inflamed and swollen. In addition, mu cus 
production occurs in the airway and muscles surroun ding 
the airw

# **Text embeddings**

In [19]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
query_result = embeddings.embed_query("Hello LLM!")

len(query_result)

384

In [21]:
query_result[:5]

[-0.01072755828499794,
 -0.027916647493839264,
 0.032464031130075455,
 0.0024128796067088842,
 0.0133972754701972]

# **VectorStore**

In [23]:
from langchain_community.vectorstores import FAISS

index = FAISS.from_documents(docs, embeddings)

## **Similar docs**

In [24]:
def get_similiar_docs(query, k=4):
    similar_docs = index.similarity_search(query, k)
    return similar_docs

In [29]:
similar_docs = get_similiar_docs("What is asthma?", 4)

for doc in similar_docs:
  print(doc.page_content)
  print("--"*100)

Introduction to Asthma 
Childhood asthma can be controlled through education and treatment. Working as a team, school 
personnel, healthcare providers and parents can help children with asthma participate fully in school, 
sports and home activities.  
What is Asthma?  
Asthma is one of the most common chronic diseases o f 
childhood, affecting more than 6 million children. 
Asthma is a chronic inflammatory lung disease that can 
cause repeated episodes of cough, wheezing and brea thing
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Asthma Triggers 
• Allergic reactions to environmental allergens such  as pollens, molds, dust mite or animal dander 
• Colds and viral respiratory infections 
• Exercise 
• Changes in weather, exposure to cold air or sudden  temperature change 
• Irritants such as tobacco smoke, air pollution, pa ints and

# **Question Answering**

In [30]:
from langchain.chains.question_answering import load_qa_chain

In [31]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model = 'mistral-small-2506',
    temperature = 0
)

In [32]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

system_prompt = """
You are an expert medical assistant. Based only on the summaries provided below,
answer the given question. Give all details as much as possible. If you don't
know the answer, respond with \"I don't know\". Do not generate new questions.
Do not make up your own answer from outside the source material.
\n\nSource material: {context}
"""

In [33]:
med_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [34]:
document_chain = create_stuff_documents_chain(llm, med_prompt)

In [35]:
rag_chain = (
    {
        "context": index.as_retriever(),
        "input": RunnablePassthrough() # Maps the input query to 'input'
    }
    | document_chain | StrOutputParser()
)

In [41]:
def get_answer(query):
    responses = rag_chain.invoke(query)
    return {"output_text": responses}

In [44]:
our_query = "Give Short answer for COVID-19 pandemiccybersecurity issues"

answer = get_answer(our_query)

print(answer['output_text'])

The COVID-19 pandemic has seen an increase in cybersecurity issues, with cybercriminals and Advanced Persistent Threat (APT) groups exploiting vulnerable people and systems. Notable issues include:

1. **APT29 targeting COVID-19 vaccine development**: APT29, a group likely based in Russia, has been observed attempting to steal COVID-19 vaccine research from various organizations involved in vaccine development.
2. **Phishing and hacking scams**: Cybercriminals are exploiting the pandemic to launch phishing attacks, often disguising themselves as government authorities or tax agencies to trick victims into clicking on malicious links related to COVID-19 assistance.
3. **Increased vulnerability**: The World Economic Forum (WEF) noted that hacking and phishing have become more effective during the pandemic due to increased anxiety and expectations of communications from authorities.

These issues highlight the need for a comprehensive approach to cybersecurity, incorporating risk manageme

In [45]:
our_query = "What are the triggers for Asthma?"

answer = get_answer(our_query)

print(answer['output_text'])

Based on the provided source material, the triggers for asthma include:

1. **Allergic reactions to environmental allergens** such as:
   - Pollens
   - Molds
   - Dust mite
   - Animal dander

2. **Respiratory infections** such as:
   - Colds
   - Viral respiratory infections

3. **Exercise** (This can also lead to Exercise-Induced Bronchoconstriction, or EIB)

4. **Changes in weather** including:
   - Exposure to cold air
   - Sudden temperature changes

5. **Irritants** such as:
   - Tobacco smoke
   - Air pollution
   - Paints
   - Cleaning agents

6. **Strong odors and/or perfumes**


In [46]:
our_query = "What is the recommended treatement for acute bronchitis?"

answer = get_answer(our_query)

print(answer['output_text'])

The recommended treatment for acute bronchitis is supportive care, as it almost always gets better on its own without antibiotics. Here are the key points:

1. **No Antibiotics**: Antibiotics are not needed to treat acute bronchitis, even if it is caused by bacteria. Using antibiotics when not needed can do more harm than good, leading to side effects like rash and diarrhea, and increasing the risk of antimicrobial resistance.

2. **Symptom Relief**: Over-the-counter medicines can help you feel better. Remember to use them as directed. Do not use cough and cold medicines in children younger than 4 years of age unless specifically told to do so by a clinician.

3. **When to Seek Medical Care**: See a healthcare professional if you or your child have:
   - Temperature higher than 100.4 °F
   - Cough with bloody mucus

4. **Prevention**: To prevent contracting bronchitis:
   - Practice good hand hygiene
   - Keep you and your child up to date with recommended vaccines

5. **Exceptions**: 

In [47]:
our_query = "List some cyber attacks that took place during the Covid-19 pandemic."

answer = get_answer(our_query)

print(answer['output_text'])

Based on the provided source material, here are some cyber-attacks that took place during the COVID-19 pandemic:

1. **Ransomware Attack on Brno University Hospital (March 2020, Czech Republic)**:
   - The Brno University Hospital, which was one of the COVID-19 testing laboratories in the country, was hit by a cyber-attack.
   - The attack forced the hospital to shut down its entire IT network.

2. **Ransomware Attack by the Maze Group (March 2020, UK)**:
   - The Maze ransomware group published personal and medical details of thousands of former patients of a London-based medical facility.
   - This attack is notable for the exposure of sensitive patient data.

3. **Phishing Scams**:
   - Cybercriminals and Advanced Persistent Threat (APT) groups launched phishing attacks targeting vulnerable people.
   - These attacks often disguised themselves as government or tax authorities, sending links to claim assistance related to COVID-19.
   - The World Economic Forum (WEF) reported that ph

In [48]:
our_query = "who is Narendra Modi?"

answer = get_answer(our_query)

print(answer['output_text'])

I don't know.
